### 安装导入依赖

In [2]:
!pip install modelscope
import cv2
from modelscope.outputs import OutputKeys
from modelscope.pipelines import pipeline
from modelscope.utils.constant import Tasks

### 加载模型

In [3]:
skin_retouching = pipeline('skin-retouching-torch',model='damo/cv_unet_skin_retouching_torch',model_revision='v1.0.4')

2025-03-10 09:08:32,176 - modelscope - INFO - Use user-specified model revision: v1.0.4


2025-03-10 09:08:33,133 - modelscope - INFO - Use user-specified model revision: v1.0.4
2025-03-10 09:08:33,508 - modelscope - WARNING - Use allow_remote=True. Will invoke codes from /mnt/workspace/.cache/modelscope/models/damo/cv_unet_skin_retouching_torch. Please make sure that you can trust the external codes.
2025-03-10 09:08:33,520 - modelscope - WARNING - ('PIPELINES', 'skin-retouching-torch', 'skin-retouching-torch') not found in ast index file
2025-03-10 09:08:33,521 - modelscope - INFO - initiate model from /mnt/workspace/.cache/modelscope/models/damo/cv_unet_skin_retouching_torch
2025-03-10 09:08:33,521 - modelscope - INFO - initiate model from location /mnt/workspace/.cache/modelscope/models/damo/cv_unet_skin_retouching_torch.
2025-03-10 09:08:33,533 - modelscope - WARNING - No preprocessor field found in cfg.
2025-03-10 09:08:33,534 - modelscope - WARNING - No val key and type key found in preprocessor domain of configuration.json file.
2025-03-10 09:08:33,534 - modelscope 

2025-03-10 09:08:36,447 - modelscope - WARNING - Model revision not specified, use revision: v2.0.2
2025-03-10 09:08:36,723 - modelscope - INFO - initiate model from /mnt/workspace/.cache/modelscope/models/damo/cv_resnet50_face-detection_retinaface
2025-03-10 09:08:36,724 - modelscope - INFO - initiate model from location /mnt/workspace/.cache/modelscope/models/damo/cv_resnet50_face-detection_retinaface.
2025-03-10 09:08:36,736 - modelscope - WARNING - No preprocessor field found in cfg.
2025-03-10 09:08:36,737 - modelscope - WARNING - No val key and type key found in preprocessor domain of configuration.json file.
2025-03-10 09:08:36,737 - modelscope - WARNING - Cannot find available config to build preprocessor at mode inference, current config: {'model_dir': '/mnt/workspace/.cache/modelscope/models/damo/cv_resnet50_face-detection_retinaface'}. trying to build by task and model information.
2025-03-10 09:08:36,737 - modelscope - WARNING - Find task: face-detection, model type: None. 

### 输入视频路径转化为图片

In [9]:
# Upload your own images
import os
from google.colab import files
import shutil

upload_folder = 'inputs'
video_path = ''

if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)
os.mkdir(upload_folder)

uploaded = files.upload()
for filename in uploaded.keys():
  dst_path = os.path.join(upload_folder, filename)
  print(f'move {filename} to {dst_path}')
  video_path = dst_path
  shutil.move(filename, dst_path)

In [ ]:
import cv2
import os

def video_to_frames(video_path, output_folder, frame_interval=1):
    # 打开视频文件
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("无法打开视频文件")
        return

    frame_count = 0
    while True:
        # 读取一帧
        ret, frame = cap.read()
        if not ret:
            break  # 视频结束

        # 保存帧为图片
        if frame_count % frame_interval == 0:
            output_path = f"{output_folder}/frame_{frame_count}.jpg"
            cv2.imwrite(output_path, frame)
            print(f"保存图片: {output_path}")

        frame_count += 1

    # 释放视频对象
    cap.release()

output_folder = "./output_frames"  # 输出图片文件夹
frame_interval = 1  # 每隔多少帧保存一次，例如 1 表示保存每一帧
if not os.path.exists(process_folder):
    os.mkdir(output_folder)

# 调用函数
video_to_frames(video_path, output_folder, frame_interval)

### 将图片美颜

In [ ]:
# 获取图片列表

process_folder = 'process_frames'
if not os.path.exists(process_folder):
    os.mkdir(process_folder)
image_list = os.listdir(output_folder)
frame_count = 1
image_list.sort()
for p in image_list:
    result = skin_retouching(f"{output_folder}/{p}")
    print(p)
    out_path = f"{process_folder}/frame_{frame_count}.jpg"
    cv2.imwrite(out_path,result[OutputKeys.OUTPUT_IMG])
    frame_count = frame_count + 1

2025-03-10 09:38:48,866 - modelscope - WARNING - task skin-retouching-torch output keys are missing


frame_0.jpg
frame_1.jpg
frame_10.jpg
frame_100.jpg
frame_101.jpg
frame_102.jpg
frame_103.jpg
frame_104.jpg
frame_105.jpg
frame_106.jpg
frame_107.jpg
frame_108.jpg
frame_109.jpg
frame_11.jpg
frame_110.jpg
frame_111.jpg
frame_112.jpg
frame_113.jpg
frame_114.jpg
frame_115.jpg
frame_116.jpg
frame_117.jpg
frame_118.jpg
frame_119.jpg
frame_12.jpg
frame_120.jpg
frame_121.jpg
frame_122.jpg
frame_123.jpg
frame_124.jpg
frame_125.jpg
frame_126.jpg
frame_127.jpg
frame_128.jpg
frame_129.jpg
frame_13.jpg
frame_130.jpg
frame_131.jpg
frame_132.jpg
frame_133.jpg
frame_134.jpg
frame_135.jpg
frame_136.jpg
frame_137.jpg
frame_138.jpg
frame_139.jpg
frame_14.jpg
frame_140.jpg
frame_141.jpg
frame_142.jpg
frame_143.jpg
frame_144.jpg
frame_145.jpg
frame_146.jpg
frame_147.jpg
frame_148.jpg
frame_149.jpg
frame_15.jpg
frame_150.jpg
frame_151.jpg
frame_152.jpg
frame_153.jpg
frame_154.jpg
frame_155.jpg
frame_156.jpg
frame_157.jpg
frame_158.jpg
frame_159.jpg
frame_16.jpg
frame_160.jpg
frame_161.jpg
frame_162.jpg
fra

### 合成视频

In [ ]:


def images_to_video(image_folder, output_video, fps=30):
    # 获取图片文件夹中的所有图片文件
    images = [img for img in os.listdir(image_folder) if img.endswith(('.png', '.jpg', '.jpeg'))]
    images.sort()  # 按文件名排序，确保顺序正确

    # 获取第一张图片的尺寸作为视频的分辨率
    first_image_path = os.path.join(image_folder, images[0])
    frame = cv2.imread(first_image_path)
    height, width, _ = frame.shape

    # 创建视频写入器
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # 视频编码格式
    video_writer = cv2.VideoWriter(output_video, fourcc, fps, (width, height))

    # 遍历图片并写入视频
    for image_name in images:
        image_path = os.path.join(image_folder, image_name)
        frame = cv2.imread(image_path)
        video_writer.write(frame)

    # 释放视频写入器
    video_writer.release()
    print(f"视频已保存到 {output_video}")

# 使用示例
image_folder = "process_frames"  # 图片文件夹路径
output_video = "output_video.mp4"  # 输出视频路径
fps = 30  # 帧率

images_to_video(image_folder, output_video, fps)